In [2]:
import pandas as pd
train_file = "train.data.txt"
dev_file = "dev.data.txt"
test_file = "test.data.txt"


train_data = pd.read_csv('./%s.csv'%train_file,keep_default_na=False)
dev_data = pd.read_csv('./%s.csv'%dev_file,keep_default_na=False)
test_data = pd.read_csv('./%s.csv'%test_file,keep_default_na=False)

In [3]:
# !pip install --upgrade tensorflow-hub
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder

# preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3")
# bert_model = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3")

le = LabelEncoder()
def preprocess_token(df, dropNa=True):
    data = df.copy()
    data['main_tweet'] = data['main_tweet'].fillna('')
    if dropNa:
        data.replace('', np.nan, inplace=True)
        data.dropna(subset=['main_tweet'], inplace=True)
    return data

train_data = preprocess_token(train_data)
train_label = le.fit_transform(train_data['label'])
dev_data = preprocess_token(dev_data)
dev_label = le.transform(dev_data['label'])

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3")
encoder_inputs = preprocessing_layer(text_input)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3", trainable=True)
outputs = encoder(encoder_inputs)
net = outputs['pooled_output']
net = tf.keras.layers.Dropout(0.1)(net)
net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
model = tf.keras.Model(inputs=text_input, outputs=net)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
train = tf.constant(train_data['main_tweet'].values.tolist())
dev = tf.constant(dev_data['main_tweet'].values.tolist())
model.fit(train, train_label, validation_data=(dev, dev_label), epochs=10, batch_size=20)
model.evaluate(dev, dev_label)

2022-05-02 00:28:45.753169: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-02 00:28:45.753985: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-02 00:28:45.754223: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: AMD Radeon Pro 555

systemMemory: 16.00 GB
maxCacheSize: 1.00 GB



2022-05-02 00:28:48.261155: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

2022-05-02 00:29:11.139023: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


79/79 [==============================] - ETA: 0s - loss: 0.9637 - accuracy: 0.7403

2022-05-02 00:34:44.356010: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


79/79 [==============================] - 456s 6s/step - loss: 0.9637 - accuracy: 0.7403 - val_loss: 0.7583 - val_accuracy: 0.7907
Epoch 2/10
79/79 [==============================] - 441s 6s/step - loss: 0.6844 - accuracy: 0.7677 - val_loss: 0.8644 - val_accuracy: 0.7626
Epoch 3/10
79/79 [==============================] - 433s 5s/step - loss: 0.7078 - accuracy: 0.6943 - val_loss: 0.5616 - val_accuracy: 0.7364
Epoch 4/10
79/79 [==============================] - 428s 5s/step - loss: 0.9112 - accuracy: 0.7147 - val_loss: 0.4975 - val_accuracy: 0.7907
Epoch 5/10
79/79 [==============================] - 427s 5s/step - loss: 0.5828 - accuracy: 0.8009 - val_loss: 0.6720 - val_accuracy: 0.7925
Epoch 6/10
79/79 [==============================] - 429s 5s/step - loss: 0.4980 - accuracy: 0.7977 - val_loss: 0.5889 - val_accuracy: 0.7925
Epoch 7/10
79/79 [==============================] - 430s 5s/step - loss: 0.4889 - accuracy: 0.8054 - val_loss: 0.5222 - val_accuracy: 0.7869
Epoch 8/10
79/79 [======

[0.5731379985809326, 0.771962583065033]